In [1]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is
from utils import helper, config, rayer, kaggle_dataset_helper

In [2]:
"""Example using LightGBM, performing randomized search with TuneSearchCV.
Example taken from https://mlfromscratch.com/gridsearch-keras-sklearn/#/
"""

from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from tune_sklearn import TuneSearchCV
import ray
from ray import tune
from sklearn import metrics
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import pandas as pd
from ray.tune.schedulers import MedianStoppingRule
from ml.models import common


In [3]:
rayer.get_global_cluster()

2022-12-02 21:39:09,060	INFO packaging.py:527 -- Creating a file package for local directory '/mnt/c/Users/rwmas/GitHub/xai/python-asd/xai/'.
2022-12-02 21:39:38,659	INFO packaging.py:354 -- Pushing file package 'gcs://_ray_pkg_dc9e6d1db2bb0664.zip' (30.53MiB) to Ray cluster...
2022-12-02 21:39:39,945	INFO packaging.py:367 -- Successfully pushed file package 'gcs://_ray_pkg_dc9e6d1db2bb0664.zip'.


In [4]:
model = BaggingClassifier()

In [5]:
ds_train, ds_test = kaggle_dataset_helper.get_transaction_predictions_dataset()

ds_train = common.label_encode(ds_train)
ds_test = common.label_encode(ds_test)

ds_train = ds_train.fillna(-1)
ds_test = ds_test.fillna(-1)

df_X = ds_train.loc[:, ds_train.columns != 'target']
df_y = ds_train['target']

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.33)# , random_state=config.rand_state)


ss = StandardScaler()
X_train_scalar = pd.DataFrame(ss.fit_transform(X_train), columns = X_train.columns)
X_test_scalar = pd.DataFrame(ss.fit_transform(X_test), columns = X_test.columns)

In [6]:
scheduler = MedianStoppingRule(grace_period=10.0)

In [6]:
param_dists  = {
    'n_estimators': tune.randint(10, 30),
}


gs = TuneSearchCV(model, param_dists, n_trials=100, scoring="accuracy", search_optimization ='hyperopt', cv=3,  
                                early_stopping=None,
                                # early_stopping=scheduler,
                                # early_stopping=True,                                
                                )
gs.fit(X_train_scalar, y_train)


ValueError: tune-sklearn uses `n_estimators` to warm start, so this parameter can't be set when warm start early stopping. 

In [ ]:
pred = gs.predict(X_test)

In [ ]:
metrics.accuracy_score(pred, y_test)

In [ ]:
# print(gs.cv_results_)

pred = gs.predict(X_test)
correct = 0
for i in range(len(y_test)):
    if pred[i] == y_test[i]:
        correct += 1
print("Accuracy:", correct / len(pred))

Accuracy: 0.9736842105263158
